In [1]:
# credits: https://www.kaggle.com/code/aatiffraz/prompt-prediction-w-mixtral-mistral7b-gemma-llama

In [2]:
!pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

MODEL_PATH = "/kaggle/input/mixtral/pytorch/8x7b-instruct-v0.1-hf/1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# To prevent GPU memory overflow in Mixtral8x7b
config = AutoConfig.from_pretrained(MODEL_PATH)
config.gradient_checkpointing = True


tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map = "auto",
    trust_remote_code = True,
    quantization_config=quantization_config,
    config=config
)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [4]:
import pandas as pd
from tqdm import tqdm
PRIVATE = True

df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/test.csv')
df.head()

,id,problem
0,000aaa,What is $1-1$?
1,111bbb,What is $0\times10$?
2,222ccc,Solve $4+x=4$ for $x$.


In [5]:
if len(df) < 5:
    df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/train.csv')
    PRIVATE = False
df.head()

,id,problem,answer
0,229ee8,"Let $k, l > 0$ be parameters. The parabola $y ...",52
1,246d26,Each of the three-digits numbers $111$ to $999...,250
2,2fc4ad,Let the `sparkle' operation on positive intege...,702
3,430b63,What is the minimum value of $5x^2+5y^2-8xy$ w...,800
4,5277ed,There exists a unique increasing geometric seq...,211


In [6]:
def gen_prompt(problem):
    
    return f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n
### Instruction:\n{problem}\n\n
### Response: Let's think step by step. The final response should be a single number in the last line of your response.
"""

In [7]:
import gc
device = 'cuda'

In [8]:
def naive_parse(answer):
    out = []
    start = False
    end = False
    for l in reversed(list(answer)):
        if l in '0123456789' and not end:
            start = True
            out.append(l)
        else:
            if start:
                end = True
        
    out = reversed(out)
    return int(''.join(out))

In [9]:
answers = []

for i in tqdm(range(len(df))):
    try:
        id_ = df['id'].loc[i]
        problem = df['problem'].loc[i]
        query_prompt = gen_prompt(problem)
        
        messages = [
            {
                "role": "user",
                "content": query_prompt
            }
        ]

        inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

        with torch.no_grad():
            encoded_output = model.generate(inputs, max_new_tokens=1500, do_sample=False, pad_token_id=tokenizer.eos_token_id)

        decoded_output = tokenizer.decode(encoded_output[0], skip_special_tokens=True).replace(query_prompt, '').replace("[INST]", "").replace("[/INST]", "").strip()
    
        print(i)
        print(decoded_output)
        
        answer = decoded_output.split('\n')[-1]
        answer = naive_parse(answer)
        print(answer)
        answer = int(answer) % 1000
        print(answer)
        answers.append(answer)
        torch.cuda.empty_cache()
        gc.collect()
        
    except Exception as e:
        print(e)
        answers.append(0)

  0%|          | 0/10 [00:00<?, ?it/s]2024-04-18 15:04:14.177288: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 15:04:14.177431: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 15:04:14.473344: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


0
First, we need to find the points of intersection $A$ and $B$ between the parabola and the line $y = 4$. This gives us the following system of equations:

$4 = kx^2 - 2kx + l$ and $y = 4 \Rightarrow 4 = y \Rightarrow 4 = kx^2 - 2kx + l$

Rearranging the second equation, we get $l = 4 - kx^2 + 2kx$. Substituting this into the first equation, we have:

$4 = kx^2 - 2kx + (4 - kx^2 + 2kx)$

Simplifying, we get:

$0 = -kx^2 + 2kx + 4$

Dividing through by $-k$, we have:

$x^2 - \frac{2}{k}x - \frac{4}{k} = 0$

This is a quadratic equation. We can solve it using the quadratic formula:

$x = \frac{-(-2/k) \pm \sqrt{(-2/k)^2 - 4(-1)(-4/k)}}{2(-1)}$

$x = \frac{2/k \pm \sqrt{4/k^2 + 16/k}}{2}$

Let $x_1$ and $x_2$ be the two solutions. Then, the points of intersection are $A(x_1, 4)$ and $B(x_2, 4)$. The sum of the squares of the distances from $A$ and $B$ to the origin is:

$d(A, 0)^2 + d(B, 0)^2 = x_1^2 + y_1^2 + x_2^2 + y_2^2 = x_1^2 + 4 + x_2^2 + 4 = x_1^2 + x_2^2 + 8$

Since the points a

 10%|█         | 1/10 [04:04<36:37, 244.18s/it]

1
To solve this problem, let's first notice that the sum of any two (not necessarily different) yellow numbers should be a blue number. This implies that the sum of any two yellow numbers should be unique, otherwise, we can replace one of them with a new blue number.

Now, let's consider the smallest and the largest possible blue numbers: 112 (the smallest even three-digit number) and 999. We can't have any yellow numbers greater than or equal to 112 because they can't form a unique sum with another yellow number without creating a blue number less than 112.

Let's see the largest possible sum of two yellow numbers that is still less than 112:
99 + 11 = 110

We can fit 9 nines and 11 ones in the given range (111-999) as yellow numbers:
99, 99, 99, 99, 99, 99, 99, 99, 11, 11, 11, ..., 11

The sum of these 12 yellow numbers is:
9 * 90 + 3 * 11 = 810 + 33 = 843

Since 843 is greater than 112, we can't fit more than 12 yellow numbers. However, we can replace one of the nines with a 112 and

 20%|██        | 2/10 [06:26<24:34, 184.30s/it]

...

 80%|████████  | 8/10 [24:22<06:22, 191.45s/it]

8
First, let's find the coordinates of points P and Q. Since A is the origin, the coordinates of P are (1/2, 0) and the coordinates of Q are (0, 1/24).

Next, we need to find the equations of the lines DP and BQ. The slope of DP is (0 - 1/2)/(1 - 0) = -1/2, so the equation of DP is y = -1/2(x - 1). The equation of BQ can be found using the fact that it passes through the point (1, 1) and has a slope of (1 - 1/24)/(0 - 1) = -1/23. So the equation of BQ is y = -1/23(x - 1).

Now we can find the points where DP and BQ intersect the horizontal and vertical lines passing through the midpoints of the sides of the square. The midpoints of the sides have coordinates (1/2, 1/2), (1/2, 0), (0, 1/2), and (1, 1/2). Solving the system of equations formed by DP and BQ with the equations of the midpoints, we find that the intersection points are (3/5, 2/5), (2/5, 3/25), (23/50, 27/50), and (27/50, 23/50).

Now we can find the areas of the four regions. The regions are bounded by the lines DP, BQ, and

 90%|█████████ | 9/10 [27:15<03:05, 185.62s/it]

9
First, let's find a general formula for $f(n)$. We know that $f(2n)=2f(n)+1$. By applying this equation recursively, we can find that $f(2^kn)=2^kf(n)+(2^k-1)$.

Now, we want to find $f(n)$ when $n=2^k$, so that $f(n)$ is as large as possible while still being a power of $2$. This happens when $k=\lfloor \log_2 n \rfloor$, so $f(100)=f(2^k)=2^kf(1)+(2^k-1)=100f(1)+99$.

Next, we need to find $f(1)$. Since $f(f(f(n)))=8n-7$, then $f(f(2))=8(2)-7=1$. Thus, $f(1)=1$ because $f$ maps positive integers to positive integers.

Now, we can find $f(100)=100f(1)+99=100(1)+99=199$. The final answer is $f(100)=199$.
199
199


100%|██████████| 10/10 [28:38<00:00, 171.82s/it]


In [10]:
df['answer'] = answers

In [11]:
df[['id','answer']].to_csv("submission.csv", header=True, index=False)

In [12]:
df[['id','answer']].head()

,id,answer
0,229ee8,2
1,246d26,13
2,2fc4ad,36
3,430b63,67
4,5277ed,880


In [13]:
if not PRIVATE:
    df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/train.csv')
    df['model_answer'] = answers
    df['match'] = df.answer == df.model_answer
    print(f'{df.match.sum()} matches in {len(df)} examples')

1 matches in 10 examples
